# Visualizing a single CT scan


Arguably, understanding the data we are presented should be the very first step in appreciating the problem at hand. I have decided to play around with files and visualizations before, for the sake of feeling alittle but of achievment, and getting a gut-instinct of what we have to deal with. I will turn my attention to perform more analytical study of a single CT scan, and my aim is to understand the format. I will also use this opportunity to build any additional utilities needed for future Model definition work.

I am very intrigued to visualize 1 CT scan in 3D. So far, we have visualized only cross-segments and projections. Let's spend some time to look up resources that will help us achieve visualizing in 3D.

- I came across a fantastic blog that will serve as the foundation for our analysis: [Vincente Rodriguez blog](https://vincentblog.xyz/posts/medical-images-in-python-computed-tomography).
- The [Wikipedia page](https://en.wikipedia.org/wiki/Marching_cubes) about `Marching cubes` is also a great introduction for 3D visualization.

## Understanding CT scans and Voxels

Before we get too far into the project, we need to take a moment to explain what a CT scan is. We have already started exploring the contents of our CT scans and as we progress we will be using data from them extensively for this project, so it is important to have a working understanding of the format we are dealing with. The key aspect we have to know is that CT scans are effectively 3D X-rays, represented as 3D arrays of single-channel data that is concatenated. It is like a stacked array of grayscale images.

A [voxel](https://en.wikipedia.org/wiki/Voxel) is a 3 dimensional representation of some value in 3D space.As with pixels in a 2D bitmap, *voxels themselves do not typically have their coordinates explicitly encoded with their values* Instead, they depend on a rendering systems to infer the position of a voxel, and we will revisit this point in a bit.

A Voxel is the 3D equivalent of a Pixel. As such, remember that:

- A Voxel does not have "width" encoded in it, yet it represents a volume - just as how a Pixel does not have a width value, until it is displayed by a renderer on either a screen or transformed to be printed. What this effectively means is that when you will slice a single Voxel from an array, you will retrieve a single point in memory that represents a volumetric point in 3D
- Voxels depend on some form of rendering system to infer their positions.

Voxels can either be cubic or they can have different shapes. To appreciate further more the data that constitutes this challenge, I provide few notes retrieved from the [LUNA 16 Challenge page](https://luna16.grand-challenge.org/data/):

|#|Descriptor | Note |
|-|-----------|------|
|1| The data used is from the [LIDC/IDRI database](http://wiki.cancerimagingarchive.net/display/Public/LIDC-IDRI) | What's this database? we need to research this. |
|2| Scans with slice thickness greater than 2.5 mm were excluded | Why were they excluded - is that something that will affect our work?|
|3| Radiologist have annotated the data: they have marked 'lesions' as non-nodules, <3 mm nodules, and >3 mm ones | What's a lesion, and how have they annotated it? Read their [publication](http://www.ncbi.nlm.nih.gov/pubmed/21452728).|
|4| Annotations | Each finding is on a line of annotations.csv. Each line contains SeriesUID and x,y,z position in **world coordinates**, and corresponding diameter in **mm**. Do we need to perform any transformation to coordinates?
|5| Objective | The LUNA16 challenge will focus on a large-scale evaluation of *automatic nodule detection* algorithms. There are two "tracks": (1) Complete systems for nodule detection, (2) systems that use a list of locations of possible nodules. The organizers provide this list to also allow teams to participate with an algorithm that only determines the likelihood for a given location in a CT scan to contain a pulmonary nodule.|



## 1 - Understanding DICOM

In previous sections, we have explored the content of the raw data but have not appreciated its structure nor did we seek to understand the meta-data. While it has been fun and it enabled us to quickly visualize some data, we need to understand what's stored in each one of those files. Remember that as a data-scientist, for you to build a model that represents the data, you must be very comfortable understanding the data that is at your hand. We should not immediately jump to any conclusion but instead wonder, with an inquisitive mind, about the possiblities of what can be achieved with what we have. To do so, I invite you to read more about the [SimpleITK](https://simpleitk.readthedocs.io/en/master/link_AdvancedImageReading_docs.html) library. Since I am not familiar with this format, I will read the documentation and revert back with notes below: